# Sphere function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

The general equation is given by:

- $f(\mathbf{x}) = \sum_{i=1}^{M} x^2_i$, with  $-\infty \le x_i \le +\infty$,

and global minimum found at:

- $f(x_1, x_2, ..., x_M) = f(0, 0, ..., 0) = 0$.

### First we import python libraries and set up the directory of our code.

In [3]:
import os, sys
import numpy as np
from math import fsum

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [5]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

### Define the sphere function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx' which takes the role of the 'random()' method of the Genes. Every time
we want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [7]:
# Sphere function.
def fun_Sphere(individual: Chromosome, f_min: bool = True):
    
    # Compute the shpere function.
    f_val = fsum([xi.value**2 for xi in individual])
    
    # Return the negative (to account for minimization).
    return -f_val if f_min else f_val
# _end_def_

# Random function: ~N(0,1).
# Since 'xi' are allowed to take any real value in [-inf, +inf], by setting this
# to the Normal(0, 1) essentially we can get both negative and positive values.
rand_fx = lambda: np.random.normal()

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by '_func'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [9]:
# Define the number of genes.
M = 5

# Define the number of chromosomes.
N = 100

# Initial population.
# It is important to note that the initial population is randomly generated with valid values.
population = [Chromosome([Gene(np.random.uniform(-10.0, +10.0), rand_fx)
                          for _ in range(M)], np.nan, True)
              for _ in range(N)]

# Create the StandardGA object that will carry on the optimization.
toy_GA = StandardGA(initial_pop=population,
                    fit_func=fun_Sphere,
                    select_op=LinearRankSelector(),
                    mutate_op=RandomMutator(),
                    cross_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
toy_GA(epochs=500, f_tol=1.0e-5, elitism=True, verbose=False)

Initial Avg. Fitness = -164.8269.
Final   Avg. Fitness = -0.1107.
Elapsed time: 5.140 seconds.


In [12]:
# Extract the optimal solution from the GA.
optimal_solution = toy_GA.best_chromosome()

# Display the (final) optimal value.
print(f"Minimum Found: {fun_Sphere(optimal_solution, f_min=False):.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.genome):
    print(f"x{i} = {xi.value:>10.6f}")
# _end_for_

# True minimum: f(0.0, 0.0, ..., 0.0) = 0.0

Minimum Found: 0.00003

x0 =   0.002938
x1 =   0.002529
x2 =  -0.002021
x3 =  -0.000153
x4 =   0.002435


**Note that the above solution is indeed very close to the absolute minimum!**

In [14]:
# If we want we can also print some operator statistics.
toy_GA.print_operator_stats()

 LinearRankSelector: (140631991771248)
 _probability: 1.0
 _counter: 500
 _lock: <unlocked _thread.lock object at 0x7fe76feaaf00>

 UniformCrossover: (140631991764688)
 _probability: 0.9
 _counter: 22456
 _lock: <unlocked _thread.lock object at 0x7fe76feab240>

 RandomMutator: (140631991562016)
 _probability: 0.1
 _counter: 5027
 _lock: <unlocked _thread.lock object at 0x7fe76feab1c0>



### End of file